In [5]:
from transformers import pipeline, AutoModelForTokenClassification, BertTokenizerFast
from absa import ABSAModel

ner_model_path = "/home/camp5/karayel/deneme_model/bert_model"

finetuned_model = AutoModelForTokenClassification.from_pretrained(ner_model_path)
tokenizer = BertTokenizerFast.from_pretrained(ner_model_path)

def response_dondur(sentence, finetuned_model=finetuned_model, tokenizer=tokenizer):

    sentence=sentence.lower()
    
    ner = pipeline("ner", model = finetuned_model, tokenizer = tokenizer, device=0)
    result = ner(sentence)

    entities = [i for i in result if i['entity']=='B-ORG']

    modelABSA = ABSAModel(tokenizer, adapter=False)

    absa_model_path = "/karayel/model_ABSA_adapter/model_lr1e-05_epochs0_batch8.pkl"

    json_1 = {}
    json_1['entity_list']=[i['word'] for i in entities]

    temp = []
    
    for count, i in enumerate(entities):
    
        x, y, z = modelABSA.predict(sentence, i['word'], load_model=absa_model_path)
    
        if y==1:
            temp.append({'entity':i['word'], 'sentiment':'olumlu'})
        elif y==0:
            temp.append({'entity':i['word'], 'sentiment':'nötr'})
        else:
            temp.append({'entity':i['word'], 'sentiment':'olumsuz'})

        if count==len(entities)-1:
            json_1['result'] = temp
            
    return json_1

In [6]:
response_dondur('Turkcell ne kadar güzel bir firma böyle')

{'entity_list': ['turkcell'],
 'result': [{'entity': 'turkcell', 'sentiment': 'olumsuz'}]}